In [30]:
#including and function
from bs4 import BeautifulSoup
import requests
import json
import re

def filter_tags(tag):
    #to find the correct tag
    if tag.name == "a" and tag.get("data-track-action") == "view article":
        return True
    elif tag.name == "p" and tag.parent.get("data-test") == "article-description":
        return True
    elif tag.name == "div" and (tag.get("data-test") == "journal-title-and-link" or tag.get("data-test") == "volume-and-page-info"):
        return True
    elif tag.name == "span" and (tag.parent.get("itemprop") == "creator" or tag.parent.get("data-test") == "article.type"):
        return True
    return False

def filter_author(tag):
    #to find author tag
    if tag.name == "a" and tag.get("data-test") == "author-name":
        return True
    else:
        return False
    
def filter_link(tag):
    #to find link
    if(tag.name=="link"):
        return True
    return False

In [31]:
#task 1
url = 'https://www.nature.com/search?q=llm&page=1'
response = requests.get(url)
response.encoding = 'utf-8'
if response.status_code == 200:
    soup  = BeautifulSoup(response.text, 'lxml')
    a_txt = soup.prettify()
    with open('page1.txt', 'w', encoding='utf-8')as file:
            file.write(f"{a_txt}\n")
            file.close()
    print('page1.txt created')
else:   
    print('failed to request the page')

page1.txt created


In [32]:
#task 2 & 3
a_tag = soup.find_all(filter_tags)
list_paper = []
current_journal = None
current_paper = None
flag = False
for i in range(len(a_tag)): 
    a = a_tag[i]
    if(a.name == "a"):
        current_paper = {'title':a.get_text(), 'url' : a.get("href"), 'author' : []}
        flag = False
    elif(a.name == "p"):
        current_paper['descirption'] = a.get_text()
    elif(a.name == "div" and a.get("data-test") == "journal-title-and-link"):
        index = a.get_text()
    elif(a.name == "div" and a.get("data-test") == "volume-and-page-info"):
        current_paper['volume-and-page-info'] = a.get_text()
    elif(a.name == "span" and a.parent.get("itemprop") == "creator"):
        current_paper['author'].append(a.get_text())
    elif(a.name == "span" and a.parent.get("data-test") == "article.type"):
        current_paper['type'] = a.get_text()
    if(i == len(a_tag) - 1 or a_tag[i+1].name == "a"):
        if current_paper.get('descirption') == None:
            current_paper['description'] = "no description"
        for jour in list_paper:
            if jour.get("journal") == index:
                jour['papers'].append(current_paper.copy())
                flag = True
                break
        if not flag:
            list_paper.append({"journal": index, "papers":[current_paper.copy()]})
        current_paper = None

for jour in list_paper:
    print(f"{jour['journal']}:{len(jour['papers'])}")

for jour in list_paper:
    for article in jour['papers']:
        url = 'https://www.nature.com'+article['url']
        response = requests.get(url)
        response.encoding = 'utf-8'
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'lxml')
        a_tag = soup.find_all(filter_author)
        author_list = []
        for a in a_tag:
            author_list.append(a.get_text())
        article['author'] = author_list.copy()
        author_list = None     

Nature Communications:8
Nature:4
Scientific Reports:10
Nature Medicine:7
Nature Machine Intelligence:8
Humanities and Social Sciences Communications:1
npj Science of Learning:1
npj Digital Medicine:5
Eye:1
Nature Methods:1
Nature Reviews Materials:1
Nature Computational Science:1
Communications Biology:1
Nature Human Behaviour:1


In [33]:
#task 4
with open('nature_llm.json', 'w', encoding='utf-8')as file:
    json.dump(list_paper, file, indent=2, separators=(',',':'))

In [35]:
#task bonus
list_paper_new = []
for jour in list_paper:
    volume_list = set()
    if 'Nature' in jour['journal']:
        for article in jour['papers']:
            volume_list.add(re.search(r'\d+', article['volume-and-page-info'], flags = 0).group())
        url = 'https://www.nature.com'+jour['papers'][0]['url']
        print(url)
        response = requests.get(url)
        response.encoding = 'utf-8'
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'lxml')
        link = soup.find_all(filter_link)
        url_1 = 'https://www.nature.com/search?volume='
        if(len(link) == 0) :
            url_2 = '&order=relevance&journal=' + jour['journal'].lower()
        else:
            url_2 = '&order=relevance&journal=' + re.search(r'com\/(\w+)', link[0].get("href"), flags = 0).group(1)
        for it in volume_list:
            url = url_1 + it + url_2
            response = requests.get(url)
            response.encoding = 'utf-8'
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'lxml')
            a_tag = soup.find_all(filter_tags)
            current_journal = None
            current_paper = None
            flag = False
            for i in range(len(a_tag)): 
                a = a_tag[i]
                if(a.name == "a"):
                    current_paper = {'title':a.get_text(), 'url' : a.get("href"), 'author' : []}
                    flag = False
                elif(a.name == "p"):
                    current_paper['descirption'] = a.get_text()
                elif(a.name == "div" and a.get("data-test") == "journal-title-and-link"):
                    index = a.get_text()
                elif(a.name == "div" and a.get("data-test") == "volume-and-page-info"):
                    current_paper['volume-and-page-info'] = a.get_text()
                elif(a.name == "span" and a.parent.get("itemprop") == "creator"):
                    current_paper['author'].append(a.get_text())
                elif(a.name == "span" and a.parent.get("data-test") == "article.type"):
                    current_paper['type'] = a.get_text()
                if(i == len(a_tag) - 1 or a_tag[i+1].name == "a"):
                    if current_paper.get('descirption') == None:
                        current_paper['description'] = "no description"
                    for jour in list_paper_new:
                        if jour.get("journal") == index:
                            jour['papers'].append(current_paper.copy())
                            flag = True
                            break
                    if not flag:
                        list_paper_new.append({"journal": index, "papers":[current_paper.copy()]})
                    current_paper = None

with open('volume.json', 'w', encoding='utf-8')as file:
    json.dump(list_paper_new, file, indent=2, separators=(',',':'))

https://www.nature.com/articles/s41467-025-57430-4
https://www.nature.com/articles/d41586-025-00662-7
https://www.nature.com/articles/s41591-024-03425-5
https://www.nature.com/articles/s42256-024-00976-7
https://www.nature.com/articles/s41592-024-02525-x
https://www.nature.com/articles/s41578-025-00772-8
https://www.nature.com/articles/s43588-025-00768-y
https://www.nature.com/articles/s41562-024-02046-9


In [36]:
print(volume_list)

{'9'}
